## 1. 什么是 ClassLoader
- 定义：**类加载器**
  - 把 class 加载到 jvm，程序可正常运行
  - jvm 启动时，并不会一次性加载所有的 class，而是选择动态选择加载（防止一次性加载太多，内存压力太大）

- 作用
  - 负责将 Class 加载到 JVM 中
  - 审查每个类由谁加载（父优先的等级加载机制）
  - 将 Class 字节码重新解析成 JVM 统一要求的对象格式

---

## 2. ClassLoader 类结构分析

```
说明：
class loader 是一个负责加载 classes 的对象，ClassLoader 类是一个抽象类，需要给出类的二进制名称，
class loader 尝试定位或者产生一个 class 的数据，
一个典型的策略是把二进制名字转换成文件名然后到文件系统中找到该文件。

/**
 * A class loader is an object that is responsible for loading classes. The
 * class ClassLoader is an abstract class.  Given the binary name of a class, a class loader should attempt to
 * locate or generate data that constitutes a definition for the class.  A
 * typical strategy is to transform the name into a file name and then read a
 * "class file" of that name from a file system.
 **/
public abstract class ClassLoader
```


```
以下是 ClassLoader 常用到的几个方法及其重载方法：
1. ClassLoader

2. defineClass(byte\[\], int, int)
// 把字节数组 btye 中的内容转换成 Java 类，返回的结果是 java.lang.Class 类的实例。这个方法被声明为 final 的
// 将 byte 字节流解析成 JVM 能够识别的 Class 对象，这个方法意味着不仅仅可以通过 class 文件实例化对象，还可以通过其他方式实例化对象，比如通过网络接收到一个类的字节码，拿到这个字节码流直接创建类的 Class 对象形式实例化对象。
// 如果直接调用这个方法生成类的 Class 对象，由于这个类的 Class 对象还没有 resolve ，这个 resolve 将会在这个对象真正实例化时才进行。

3. findClass(String name)
// 查找名称为 name 的类，返回的结果是 java.lang.Class 类的实例

4. loadClass(String name)
// 加载名称为 name 的类，返回的结果是 java.lang.Class 类的实例

5. resolveClass(Class<?>)
// 链接指定的 Java 类
```

### 2.1. loadClass 方法
- 使用指定的二进制名称来加载类，执行顺序如下
  - 调用 findLoadedClass(String) 方法检查这个类是否被加载过
  - 使用父加载器调用 loadClass(String) 方法
  - 如果父加载器为 Null，类加载器装载虚拟机内置的加载器调用 findClass(String) 方法装载类
  - 如果找到了类，并且该方法接收的 resolve 参数的值为 true，那么就调用 resolveClass(Class) 方法来处理类
  - **除非被重写，这个方法默认在整个装载过程中都是同步的（线程安全的）**

```
protected Class> loadClass(String name, boolean resolve) throws ClassNotFoundException {
    synchronized (getClassLoadingLock(name)) {
        // First, check if the class has already been loaded
        Class c = findLoadedClass(name);
        if (c == null) {
            long t0 = System.nanoTime();
            try {
                if (parent != null) {
                    c = parent.loadClass(name, false);
                } else {
                    c = findBootstrapClassOrNull(name);
                }
            } catch (ClassNotFoundException e) {
                // ClassNotFoundException thrown if class not found
                // from the non-null parent class loader
            }

            if (c == null) {
                // If still not found, then invoke findClass in order
                // to find the class.
                long t1 = System.nanoTime();
                c = findClass(name);

                // this is the defining class loader; record the stats
                sun.misc.PerfCounter.getParentDelegationTime().addTime(t1 - t0);
                sun.misc.PerfCounter.getFindClassTime().addElapsedTimeFrom(t1);
                sun.misc.PerfCounter.getFindClasses().increment();
            }
        }
        if (resolve) {
            resolveClass(c);
        }
        return c;
    }
}
```

---

## 3. ClassLoader 的等级加载机制
- Java 默认提供的三个 ClassLoader

### 3.1. BootStrap ClassLoader
- 启动类加载器，是 Java 类加载层次中最顶层的类加载器
- 负责加载 JDK 中的核心类库，如：rt.jar、resources.jar、charsets.jar 等

```
// 获得该类加载器从哪些地方加载了相关的 jar 或 class 文件
public class BootStrapTest {
    public static void main(String[] args) {
      URL[] urls = sun.misc.Launcher.getBootstrapClassPath().getURLs();
      for (int i = 0; i < urls.length; i++) {
          System.out.println(urls[i].toExternalForm());
       }
    }
}

// 输出结果
file:/C:/Program%20Files/Java/jdk1.6.0_22/jre/lib/resources.jar
file:/C:/Program%20Files/Java/jdk1.6.0_22/jre/lib/rt.jar
file:/C:/Program%20Files/Java/jdk1.6.0_22/jre/lib/sunrsasign.jar
file:/C:/Program%20Files/Java/jdk1.6.0_22/jre/lib/jsse.jar
file:/C:/Program%20Files/Java/jdk1.6.0_22/jre/lib/jce.jar
file:/C:/Program%20Files/Java/jdk1.6.0_22/jre/lib/charsets.jar
file:/C:/Program%20Files/Java/jdk1.6.0_22/jre/classes/


// 其实上述结果也是通过查找 sun.boot.class.path 这个系统属性所得知的
System.out.println(System.getProperty("sun.boot.class.path"));
```

### 3.2. Extension ClassLoader
- 扩展类加载器
- 负责加载 Java 的扩展类库，Java 虚拟机的实现会提供一个扩展库目录
- 该类加载器在此目录里面查找并加载 Java 类
- 默认加载 JAVA_HOME/jre/lib/ext/ 目录下的所有 jar

### 3.3. App ClassLoader
- 系统类加载器
- 负责加载应用程序 classpath 目录下的所有 jar 和 class 文件
- 一般来说，Java 应用的类都是由它来完成加载的
- 通过 ClassLoader.getSystemClassLoader() 来获取它

### 3.4. 其他
- 开发人员可以通过继承 java.lang.ClassLoader 类的方式实现自己的类加载器
- 这些自定义的 ClassLoader 都必须继承自 java.lang.ClassLoader 类，也包括 Java 提供的另外二个 ClassLoader
- 但是 **Bootstrap ClassLoader 不继承自 ClassLoader**，因为它不是一个普通的 Java 类，底层由 C++ 编写，已嵌入到了 JVM 内核当中，当 JVM 启动后，Bootstrap ClassLoader 也随着启动，负责加载完核心类库后，并构造 Extension ClassLoader 和 App ClassLoader 类加载器。

---

## 4. ClassLoader 加载类的原理
### 4.1. 原理介绍
- ClassLoader 使用的是双亲委托模型来搜索类的，每个 ClassLoader 实例都有一个父类加载器的引用(不是继承的关系，是一个包含的关系)
  - 虚拟机内置的类加载器(Bootstrap ClassLoader)本身没有父类加载器，但可以用作其它 ClassLoader 实例的的父类加载器。

> 当一个 ClassLoader 实例需要加载某个类时，它会试图亲自搜索某个类之前，先把这个任务委托给它的父类加载器，这个过程是由上至下依次检查的，首先由最顶层的类加载器 Bootstrap ClassLoader 试图加载，如果没加载到，则把任务转交给 Extension ClassLoader 试图加载，如果也没加载到，则转交给 App ClassLoader 进行加载，如果它也没有加载得到的话，则返回给委托的发起者，由它到指定的文件系统或网络等 URL 中加载该类。

> 如果它们都没有加载到这个类时，则抛出 ClassNotFoundException 异常。否则将这个找到的类生成一个类的定义，并将它加载到内存当中，最后返回这个类在内存中的 Class 实例对象。

### 4.2. 为什么要使用双亲委托这种模型呢？
- **避免重复加载**：当父亲已经加载了该类的时候，就没有必要 ClassLoader 再加载一次
- **安全因素**：如果不使用委托模式，那我们就可以随时使用自定义的 String 来动态替代 java 核心 api 中定义的类型，这样会存在非常大的安全隐患。

### 4.3 JVM在搜索类的时候，如何判定两个 class 是相同的呢？
- JVM 判定两个 class 是否相同，需要同时满足两个条件
  - 判断两个类名是否相同
  - 判断是否由同一个类加载器实例加载的

---

## 5. ClassLoader 架构体系
![ClassLoader的体系架构](ClassLoader的体系架构.png)

### 5.1 类加载器的树状组织结构
- 每个 Java 类都维护着一个指向定义它的类加载器的引用，通过 getClassLoader() 方法获取引用

```
public class ClassLoaderTree {
    public static void main(String[] args) {
        ClassLoader loader = ClassLoaderTree.class.getClassLoader();
        while (loader != null) {
            System.out.println(loader.toString());
            loader = loader.getParent();
        }
        System.out.println(loader);
    }
}

// 第一个输出的是 ClassLoaderTree 类的类加载器，即系统类加载器
sun.misc.Launcher$AppClassLoader@42a45678
// 第二个输出的是扩展类加载器
sun.misc.Launcher$ExtClassLoader@74a12323
// 第三个输出的 null
// ExtClassLoader 的 类加器是 Bootstrap ClassLoader，因为 Bootstrap ClassLoader 不是一个普通的 Java 类
// 所以 ExtClassLoader 的 parent = null
null
```

### 5.2 JVM 加载 class 文件的两种方法
- **隐式加载**:
  - 程序在运行过程中当碰到通过 new 等方式生成对象时，隐式调用类装载器加载对应的类到 jvm 中
- **显式加载**: 
  - 通过 class.forname()、this.getClass.getClassLoader().loadClass() 等方法显式加载需要的类，或者我们自己实现的 ClassLoader 的 findlass() 方法

---

## 6. 如何加载 class 文件
1. 找到 .class 文件并把这个文件包含的字节码加载到内存中
2. 分三步，字节码验证；class 类数据结构分析及相应的内存分配；最后的符号表的链接
3. 类中静态属性和初始化赋值，以及静态块的执行等

![jvm加载类步骤](jvm加载类步骤.png)

### 6.1 验证与分析
- 字节码验证，类装入器对于类的字节码要做很多检测，以确保格式正确，行为正确
- 类装备，准备代表每个类中定义的字段、方法和实现接口所必须的数据结构
- 解析，装入器装入类所引用的其他所有类

---

## 7. 常见加载类错误分析
### 7.1 ClassNotFoundExecption
- 通常发生在显示加载类的时候
  - 通过类 Class 中的 forName() 方法
  - 通过类 ClassLoader 中的 loadClass() 方法
  - 通过类 ClassLoader 中的 findSystemClass() 方法

### 7.2 NoClassDefFoundError
- 可能的情况就是使用 new 关键字、属性引用某个类、继承了某个接口或者类，以及方法的某个参数中引用了某个类，这时就会触发 JVM 或者类加载器实例尝试加载类型的定义，但是该定义却没有找到，影响了执行路径。
- 在编译时这个类是能够被找到的，但是在执行时却没有找到。

### 7.3 UnsatisfiedLinkError
- 该错误通常是在 JVM 启动的时候，如果 JVM 中的某个 lib 删除了，就有可能报这个错误
```
public class UnsatisfiedLinkErrorTest{
    public native void nativeMethod();
    static {
        System.loadLibrary("NoLib");
    }
    public static void main(String[] args) {
        // 解析 native 标识的方法时 JVM 找不到对应的库文件
        new UnsatisfiedLinkErrorTest().nativeMethod();
    }
}
```

### 7.4 ClassCastException
- 该错误通常出现强制类型转换时出现这个错误
- 注意：JVM 在做类型转换时的规则：
  - 对于普通对象，对象必须是目标类的实例或目标类的子类的实例。如果目标类是接口，那么会把它当作实现了该接口的一个子类
  - 对于数组类型，目标类必须是数组类型或 java.lang.Object、java.lang.Cloneable、java.io.Serializable
- 如果不满足上面的规则，JVM 就会报错，有两种方式可避免错误
  - 在容器类型中显式的指明这个容器所包含的对象类型
  - 先通过 instanceof 检查是不是目标类型，然后再进行强制类型的转换

### 7.5 ExceptionInInitializerError
- 在初始化这个类时，给静态属性 m 赋值时出现了异常

```
public class ExceptionInInitializerErrorTest {
    public static Map m = new HashMap(){{
        // 使用 m 会报错
        m.put("a", "2");
    }};
    public static void main(String[] args) {
        Integer integer = (Integer) m.get("a");
        System.out.println(integer);
    }
}
```

### 7.6 NoSuchMethodError
- 未找到方法

### 7.7 LinkageError
- 和 ClassCastException 本质一样，加载自不同位置的相同类在同一段逻辑（比如：方法）中交互时，会出现 LinkageError
- 需要观察哪个类被不同的类加载器加载了，在哪个方法或者调用处发生（交汇）的，解决方法无外乎两种
  - 还是不同的类加载器加载，但是相互不再交汇影响，这里需要针对发生问题的地方做一些改动，比如更换实现方式
  - 冲突的类需要由一个 Parent 类加载器进行加载

---

## 8. 如何实现自己的 ClassLoader
- ClassLoader 能够完成的事情有以下情况
  - 在自定义路径下查找自定义的 class 类文件
  - 对我们自己要加载的类做特殊处理
  - 可以定义类的实现机制

- 定义自已的类加载器分为两步
  - 1. 继承 java.lang.ClassLoader
  - 2. 重写父类的 findClass 方法

### 8.1 文件系统类加载器
- 为了保证类加载器都正确实现代理模式，在开发自己的类加载器时，最好不要覆写 loadClass() 方法，而是覆写 findClass() 方法

```
public class FileSystemClassLoader extends ClassLoader {
    private String rootDir;

    public FileSystemClassLoader(String rootDir){
        this.rootDir = rootDir;
    }

    protected Class<?> findClass(String name) throws ClassNotFoundException {
        byte[] classData = getClassData(name);
        if (classData == null){
            throw new ClassNotFoundException();
        }
        else {
            return defineClass(name, classData, 0, classData.length);
        }
    }

    private byte[] getClassData(String className) {
        String path = classNameToPath(className);
        try {
            InputStream ins = new FileInputStream(path);
            ByteArrayOutputStream baos = new ByteArrayOutputStream();
            int bufferSize = 4096;
            byte[] buffer = new byte[bufferSize];
            int bytesNumRead = 0;
            while ((bytesNumRead = ins.read(buffer)) != -1){
                baos.write(buffer, 0, bytesNumRead);
            }
            return baos.toByteArray();
        } catch (FileNotFoundException e) {
            e.printStackTrace();
        } catch (IOException e) {
            e.printStackTrace();
        }
        return null;
    }

    private String classNameToPath(String className) {
        return rootDir + File.separatorChar + className.replace('.', File.separatorChar) + ".class";
    }
}
```

### 8.2 网络类加载器
- 需要注意的是，并不能直接在客户端代码中引用从服务器上下载的类，因为客户端代码的类加载器找不到这些类。
- 一般有两种做法来使用它
  - 一种做法是使用 Java 反射 API
    - 使用 Java 反射 API 可以直接调用 Java 类的方法
  - 一种做法是使用接口
    - 使用接口的做法则是把接口的类放在客户端中，从服务器上加载实现此接口的不同版本的类
    - 在客户端通过相同的接口来使用这些实现类

---

## 9. 类加载器与 Web 容器
- 对于运行在 Java EE™ 容器中的 Web 应用来说，类加载器的实现方式与一般的 Java 应用有所不同。不同的 Web 容器的实现方式也会有所不同。
- 以 Apache Tomcat 来说，每个 Web 应用都有一个对应的类加载器实例。该类加载器也使用代理模式，所不同的是它是首先尝试去加载某个类，如果找不到再代理给父类加载器。这与一般类加载器的顺序是相反的。
- 这是 Java Servlet 规范中的推荐做法，其目的是使得 Web 应用自己的类的优先级高于 Web 容器提供的类。
- 这种代理模式的一个例外是：Java 核心库的类是不在查找范围之内的。这也是为了保证 Java 核心库的类型安全。

---